## CS591 Homework 2
#### By Anton Njavro

### EDA (Exploratory data analysis)

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [40]:
#Load data
websiteA = pd.read_csv('WebsiteA.csv')
websiteB = pd.read_csv('WebsiteB.csv')

#Rename columns
websiteA.columns = ['TimeStamp','DwellTime','Converted','PurchaseAmount']
websiteB.columns = ['TimeStamp','DwellTime','Converted','PurchaseAmount']

In [41]:
#Display data
websiteA.head()


,TimeStamp,DwellTime,Converted,PurchaseAmount
0,179,58,0,0
1,307,55,0,0
2,540,97,0,0
3,788,16,0,0
4,804,553,0,0


In [55]:
#Get numbers on number of converts and amounts
number_of_A_converts = np.sum(websiteA['Converted'])
number_of_B_converts = np.sum(websiteB['Converted'])

#Convert ratio
convert_ratio_A = round(number_of_A_converts/len(websiteA),4)
convert_ratio_B = round(number_of_B_converts/len(websiteB),4)

#Revenue per customer 
rpcA =  round(np.sum(websiteA['PurchaseAmount'])/number_of_A_converts,2)
rpcB =  round(np.sum(websiteB['PurchaseAmount'])/number_of_B_converts,2)

print("A converts: ", number_of_A_converts)
print("B converts: ", number_of_B_converts)
print()
print("A convert ratio: ", convert_ratio_A)
print("B convert ratio: ", convert_ratio_B)
print()
print("Revenue per customer (RPC) on website A: ", rpcA)
print("Revenue per customer (RPC) on website B: ", rpcB)

A converts:  905
B converts:  800

A convert ratio:  0.0905
B convert ratio:  0.1

Revenue per customer (RPC) on website A:  414.14
Revenue per customer (RPC) on website B:  381.67


In [ ]:
#Let's plot histograms
_ = plt.hist(x = websiteA[''], y = )